In [26]:
from time import sleep
import random
import re
import pandas as pd
from selenium import webdriver
import sys
import cx_Oracle
# 라이브러리 임포트
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Text
from sqlalchemy.dialects.oracle import NUMBER, VARCHAR2

In [43]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Text
from sqlalchemy.dialects.oracle import NUMBER, VARCHAR2

# 데이터 베이스 연결 정보 설정
db_username = 'base_man'
db_password = '1111'
db_host = '192.168.0.29' # 다른 IP 주소
db_port = '1521' #Oracle의 기본포트
service_name = 'XE'

engine = create_engine(f'oracle+cx_oracle://{db_username}:{db_password}@{db_host}:{db_port}/?service_name={service_name}')

# Oracle 데이터베이스 연결 문자열 생성
dsn_tns = cx_Oracle.makedsn(db_host, db_port, service_name=service_name)

# 데이터베이스 연결
connection = cx_Oracle.connect(user=db_username, password=db_password, dsn=dsn_tns)

# 커서 생성
cursor = connection.cursor()

In [6]:
#db에서 가져옴
query = "select * from emotion2"
ef = pd.read_sql(query, con=engine)

In [12]:
emdf = pd.DataFrame(ef)

In [13]:
emdf.head(1)

,제목,댓글내용,감성분석,감성점수,링크,썸네일,team_code,좋아요,조회수
0,"8월 3번째 스윕승, 4연승, 승리 직후의 모습#애프터매치(08.29 키움전)",우리의 지금 순위는 구자욱이 주장인게 진짜 의미가 있다... 잘하는 선수 바로바로 ...,긍정적,126.0,https://www.youtube.com/watch?v=4CEvMrPw5xw,https://img.youtube.com/vi/4CEvMrPw5xw/maxresd...,SS,2100,46000


In [46]:
#가중치 설정
sentiment_weight = 0.5 #감성점수
likes_weight = 0.3 #좋아요 점수
views_weight = 0.2 #조회수 점수

#추천 점수 계산

emdf['추천점수']=(
    (emdf['감성점수'] * sentiment_weight)+
    (emdf['좋아요'] * likes_weight) +
    (emdf['조회수'] * views_weight)
)

# 새로운 데이터 프레임생성
nemdf = emdf[['제목','댓글내용','감성분석','감성점수','링크','썸네일','team_code','좋아요','조회수','추천점수']]
nemdf

,제목,댓글내용,감성분석,감성점수,링크,썸네일,team_code,좋아요,조회수,추천점수
0,"8월 3번째 스윕승, 4연승, 승리 직후의 모습#애프터매치(08.29 키움전)",우리의 지금 순위는 구자욱이 주장인게 진짜 의미가 있다... 잘하는 선수 바로바로 ...,긍정적,126.0,https://www.youtube.com/watch?v=4CEvMrPw5xw,https://img.youtube.com/vi/4CEvMrPw5xw/maxresd...,SS,2100,46000,9893.00
1,"하늘엔 수많은 색깔의 별이 있잖아. 그 중에 가장 뜨거운 빛을 뿜어 내는 건, 파랑...",“ 유격수 이재현이 더이상의 실점을 허락하지 않습니다 ” 정우영 캐스터가 중계한 멘...,긍정적,124.0,https://www.youtube.com/watch?v=Qic3h--TfRA,https://img.youtube.com/vi/Qic3h--TfRA/maxresd...,SS,2500,77000,16212.00
2,영원히 야구장에 푸른 파도가 멈추지 않도록 📹 #덕아웃1열 (08.29 키움전),"아마도 22년 2월. 구자욱의 블로그에서 감동적으로 읽고 기억해 두었던 그 글귀 ""...",긍정적,94.0,https://www.youtube.com/watch?v=lgBk9wbil40,https://img.youtube.com/vi/lgBk9wbil40/maxresd...,SS,2700,88000,18457.00
3,"WIN and WOW, 3연승, 승리 직후의 모습#애프터매치(08.28 키움전)",아니이제 애프터매치도 낯안가리는 박뱅이라니...... 귀하다귀해 인터뷰 안 피하는 ...,긍정적,74.5,https://www.youtube.com/watch?v=PWil5aTn7e4,https://img.youtube.com/vi/PWil5aTn7e4/maxresd...,SS,1900,42000,9007.25
4,"데뷔 첫 완봉승, 천금의 한 점, 승리 직후의 모습 #애프터매치(08.27 키움전)",갑작스럽게 나오게 되었지만 늘 꼭 필요한 순간에 한방 터트려주는 안경 선배 멋있어요...,긍정적,53.0,https://www.youtube.com/watch?v=QkMRf7lvbe0,https://img.youtube.com/vi/QkMRf7lvbe0/maxresd...,SS,2600,65000,13806.50
5,✨삼성라이온즈✨ 평점 10/10 도파민 터지는 삼성라이온즈 야구를 추천하오. 소인은...,정빈이는 진짜 유명한 ㅈ댐감지기가 맞다 ㅋㅋㅋㅋㅋㅋㅋ진짜 벼랑끝에 왔을때 하나씩 해...,긍정적,36.5,https://www.youtube.com/watch?v=qergpNg580U,https://img.youtube.com/vi/qergpNg580U/maxresd...,SS,2400,74000,15538.25
6,"""인성이 여기서 나타나는 거야"" ""우리 지광이는요..평양냉면을 좋아해요"" T와 F적...",지찬아 너 삼성이랑 20년 계약했어… 기억 안 나 ? 태인아 너 비fa다년계약 20...,긍정적,21.5,https://www.youtube.com/watch?v=NjrZdLiEHYs,https://img.youtube.com/vi/NjrZdLiEHYs/maxresd...,SS,2500,81000,16960.75
7,(월간라클) 팬 vs 선수 라이온즈 푸른 피 TEST✏ #월간라클,2:12 좌승보다 지찬이가 형이라니...깜짝놀랐네.. 1:23 영웅이 협찬 해달라는...,긍정적,19.0,https://www.youtube.com/watch?v=BRnQTmWaV_A,https://img.youtube.com/vi/BRnQTmWaV_A/maxresd...,SS,2000,62000,13009.50


In [48]:
# 테이블 생성 쿼리
create_table_query = '''
create table EMOTION3(
   제목 VARCHAR2(300),
   댓글내용 CLOB,
   감성분석 VARCHAR2(50),
   감성점수 NUMBER,
   링크 VARCHAR2(225),
   썸네일 VARCHAR2(225),
   team_code VARCHAR2(50),
   좋아요 NUMBER,
   조회수 NUMBER,
   추천점수 NUMBER
)
'''
# 쿼리 실행
cursor.execute(create_table_query)

#변경 사항 커밋
connection.commit()

#커서와 연결 종료
cursor.close()
connection.close()

#성공 알람
print("Table created successfully!!")

Table created successfully!!


In [49]:
engine = create_engine(f'oracle+cx_oracle://{db_username}:{db_password}@{db_host}:{db_port}/?service_name={service_name}')
nemdf.to_sql(
    'EMOTION3', 
    con=engine, 
    if_exists='append', 
    index=False, 
    dtype={
        '제목' :VARCHAR2(300),
        '댓글내용': Text(),
        '감성분석': VARCHAR2(50),
        '감성점수': NUMBER,
        '링크': VARCHAR2(225),
        '썸네일': VARCHAR2(225),
        'team_code': VARCHAR2(50),
        '좋아요': NUMBER,
        '조회수': NUMBER,
        '추천점수':NUMBER
    }
)
print("Table insert successfully")

Table insert successfully


C:\Users\user\AppData\Local\Temp\ipykernel_4596\3987161025.py:2: UserWarning: The provided table name 'EMOTION3' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  nemdf.to_sql(
